In [5]:
query_male = '''PREFIX icon: <https://w3id.org/icon/ontology/>
PREFIX iig: <https://w3id.org/iicongraph/data/>
PREFIX sim: <https://w3id.org/simulation/ontology/>
SELECT ?element ?femalePercentage ?malePercentage (?malePercentage - ?femalePercentage AS ?difference) (?malePercentage / ?femalePercentage AS ?ratio)
WHERE {
  {
    SELECT ?element (ROUND((COUNT(?artwork) / ?femaleTotal) * 100 * 1000) /1000 AS ?femalePercentage)
    WHERE {
      {
        SELECT (COUNT(DISTINCT ?artwork) AS ?femaleTotal)
        WHERE {
          ?artwork icon:preiconographicallyDepicts ?subject .
          FILTER (?subject IN (iig:femaleFigures, iig:femaleFigure))
          FILTER NOT EXISTS {
            ?artwork icon:preiconographicallyDepicts ?maleSubject .
            FILTER (?maleSubject IN (iig:maleFigures, iig:maleFigure))
          }
        }
      }
      ?artwork icon:preiconographicallyDepicts ?subject .
      FILTER (?subject IN (iig:femaleFigures, iig:femaleFigure))
      FILTER NOT EXISTS {
        ?artwork icon:preiconographicallyDepicts ?maleSubject .
        FILTER (?maleSubject IN (iig:maleFigures, iig:maleFigure))
      }
      ?artwork ?predicate ?element .
      FILTER (?predicate IN (icon:preiconographicallyDepicts, icon:iconographicallyDepicts, icon:iconologicallyRepresents))
      FILTER (?element NOT IN (iig:femaleFigures, iig:femaleFigure))
            FILTER NOT EXISTS {?element sim:hasSimulacrum ?something}
    }
    GROUP BY ?element ?femaleTotal
  }
  {
    SELECT ?element (ROUND((COUNT(?artwork) / ?maleTotal) * 100 *1000) /1000 AS ?malePercentage)
    WHERE {
      {
        SELECT (COUNT(DISTINCT ?artwork) AS ?maleTotal)
        WHERE {
          ?artwork icon:preiconographicallyDepicts ?subject .
          FILTER (?subject IN (iig:maleFigures, iig:maleFigure))
          FILTER NOT EXISTS {
            ?artwork icon:preiconographicallyDepicts ?femaleSubject .
            FILTER (?femaleSubject IN (iig:femaleFigures, iig:femaleFigure))
          }
        }
      }
      ?artwork icon:preiconographicallyDepicts ?subject .
      FILTER (?subject IN (iig:maleFigures, iig:maleFigure))
      FILTER NOT EXISTS {
        ?artwork icon:preiconographicallyDepicts ?femaleSubject .
        FILTER (?femaleSubject IN (iig:femaleFigures, iig:femaleFigure))
      }
      ?artwork ?predicate ?element .
      FILTER (?predicate IN (icon:preiconographicallyDepicts, icon:iconographicallyDepicts, icon:iconologicallyRepresents))
      FILTER (?element NOT IN (iig:maleFigures, iig:maleFigure))
            FILTER NOT EXISTS {?element sim:hasSimulacrum ?something}
    }
    GROUP BY ?element ?maleTotal
  }
  FILTER (BOUND(?femalePercentage) && BOUND(?malePercentage))
  FILTER (?malePercentage > ?femalePercentage)  # Now filtering for cases where male % is higher
}
ORDER BY DESC(?difference) #Change this to ORDER BY DESC(?difference) to order by the difference in percentage
LIMIT 20'''


In [6]:
query_female = '''PREFIX icon: <https://w3id.org/icon/ontology/>
PREFIX iig: <https://w3id.org/iicongraph/data/>
PREFIX sim: <https://w3id.org/simulation/ontology/>
SELECT ?element ?femalePercentage ?malePercentage (?femalePercentage - ?malePercentage AS ?difference) (?femalePercentage / ?malePercentage AS ?ratio)
WHERE {
  {
    SELECT ?element (ROUND((COUNT(?artwork) / ?femaleTotal) * 100 * 1000) /1000 AS ?femalePercentage)
    WHERE {
      {
        SELECT (COUNT(DISTINCT ?artwork) AS ?femaleTotal)
        WHERE {
          ?artwork icon:preiconographicallyDepicts ?subject .
          FILTER (?subject IN (iig:femaleFigures, iig:femaleFigure))
          FILTER NOT EXISTS {
            ?artwork icon:preiconographicallyDepicts ?maleSubject .
            FILTER (?maleSubject IN (iig:maleFigures, iig:maleFigure))
          }
        }
      }
      ?artwork icon:preiconographicallyDepicts ?subject .
      FILTER (?subject IN (iig:femaleFigures, iig:femaleFigure))
      FILTER NOT EXISTS {
        ?artwork icon:preiconographicallyDepicts ?maleSubject .
        FILTER (?maleSubject IN (iig:maleFigures, iig:maleFigure))
      }
      ?artwork ?predicate ?element .
      FILTER (?predicate IN (icon:preiconographicallyDepicts, icon:iconographicallyDepicts, icon:iconologicallyRepresents))
      FILTER (?element NOT IN (iig:femaleFigures, iig:femaleFigure))
            FILTER NOT EXISTS {?element sim:hasSimulacrum ?something}
    }
    GROUP BY ?element ?femaleTotal
  }
  {
    SELECT ?element (ROUND((COUNT(?artwork) / ?maleTotal) * 100 *1000) / 1000 AS ?malePercentage)
    WHERE {
      {
        SELECT (COUNT(DISTINCT ?artwork) AS ?maleTotal)
        WHERE {
          ?artwork icon:preiconographicallyDepicts ?subject .
          FILTER (?subject IN (iig:maleFigures, iig:maleFigure))
          FILTER NOT EXISTS {
            ?artwork icon:preiconographicallyDepicts ?femaleSubject .
            FILTER (?femaleSubject IN (iig:femaleFigures, iig:femaleFigure))
          }
        }
      }
      ?artwork icon:preiconographicallyDepicts ?subject .
      FILTER (?subject IN (iig:maleFigures, iig:maleFigure))
      FILTER NOT EXISTS {
        ?artwork icon:preiconographicallyDepicts ?femaleSubject .
        FILTER (?femaleSubject IN (iig:femaleFigures, iig:femaleFigure))
      }
      ?artwork ?predicate ?element .
      FILTER (?predicate IN (icon:preiconographicallyDepicts, icon:iconographicallyDepicts, icon:iconologicallyRepresents))
      FILTER (?element NOT IN (iig:maleFigures, iig:maleFigure))
            FILTER NOT EXISTS {?element sim:hasSimulacrum ?something}
    }
    GROUP BY ?element ?maleTotal
  }
  FILTER (BOUND(?femalePercentage) && BOUND(?malePercentage))
  FILTER (?femalePercentage > ?malePercentage)  # Now filtering for cases where female % is higher
}
ORDER BY DESC(?ratio) #Change this to ORDER BY DESC(?difference) to order by the difference in percentage
LIMIT 25
'''

In [28]:
from pymantic import sparql


blaze_or_gdb_url = "http://localhost:7200/repositories/IICONGRAPHarco" # fill this with the url of blazegraph or graphbd, blazegraph will look similar to this 'http://192.168.1.9:9999/bigdata/sparql'
# graphdb will look similar to 'http://localhost:7200/repositories/IDOFYOURIICONGRAPHarcoREPOSITORY'
server = sparql.SPARQLServer(blaze_or_gdb_url) #only works if you set up an instance of blazegraph/graphdb and load iicongraph-arco.ttl into it.



results_man = server.query(query_male)
result_women = server.query(query_female)

In [32]:
for row in results_man["results"]["bindings"]:
    print(row["element"]["value"], row["femalePercentage"]["value"], row["malePercentage"]["value"], row["difference"]["value"], round(float(row["ratio"]["value"]), 5))

https://w3id.org/iicongraph/data/promotionOfSport 4.582 14.91 10.328 3.25404
https://w3id.org/iicongraph/data/promotionOfInformationAndCommunication 9.319 18.27 8.951 1.96051
https://w3id.org/iicongraph/data/promotionOfCompetitions 2.174 8.505 6.331 3.91214
https://w3id.org/iicongraph/data/promotionOfPolitics 2.692 6.972 4.280 2.5899
https://w3id.org/iicongraph/data/horse 0.673 4.746 4.073 7.05201
https://w3id.org/iicongraph/data/horses 0.233 4.284 4.051 18.38627
https://w3id.org/iicongraph/data/promotionOfWar 0.699 4.578 3.879 6.54936
https://w3id.org/iicongraph/data/promotionOfPoliticalParties 1.32 4.935 3.615 3.73864
https://w3id.org/iicongraph/data/promotionOfPublicAdministration 4.064 7.644 3.580 1.88091
https://w3id.org/iicongraph/data/promotionOfSportsAssociations 0.285 3.549 3.264 12.45263
https://w3id.org/iicongraph/data/promotionOfHorseRacing 0.595 3.801 3.206 6.38824
https://w3id.org/iicongraph/data/promotionOfFascistPropaganda 0.699 3.507 2.808 5.01717
https://w3id.org/iico

In [35]:
for row in result_women["results"]["bindings"]:
    print(row["element"]["value"], row["femalePercentage"]["value"], row["malePercentage"]["value"], row["difference"]["value"], round(float(row["ratio"]["value"]), 5))

https://w3id.org/iicongraph/data/peasant 1.165 0.021 1.144 55.47619
https://w3id.org/iicongraph/data/promotionOfLaundryDetergent 2.174 0.042 2.132 51.7619
https://w3id.org/iicongraph/data/veil 1.035 0.021 1.014 49.28571
https://w3id.org/iicongraph/data/barOfSoap 1.035 0.021 1.014 49.28571
https://w3id.org/iicongraph/data/promotionOfSewingMachine 0.828 0.021 0.807 39.42857
https://w3id.org/iicongraph/data/swimmingSuit 2.459 0.063 2.396 39.03175
https://w3id.org/iicongraph/data/jewels 0.751 0.021 0.730 35.7619
https://w3id.org/iicongraph/data/actress 1.45 0.042 1.408 34.52381
https://w3id.org/iicongraph/data/promotionOfCareFacilities 0.673 0.021 0.652 32.04762
https://w3id.org/iicongraph/data/promotionOfSoapsAndDetergents 3.339 0.105 3.234 31.8
https://w3id.org/iicongraph/data/fur 0.647 0.021 0.626 30.80952
https://w3id.org/iicongraph/data/promotionOfShampoo 0.595 0.021 0.574 28.33333
https://w3id.org/iicongraph/data/bather 0.57 0.021 0.549 27.14286
https://w3id.org/iicongraph/data/promo